This notebook is to generate a fake gff file to understand the best strategy to merge two annotations

https://github.com/The-Sequence-Ontology/Specifications/blob/master/gff3.md

In [18]:
import pyranges as pr
import numpy as np
import random
import pandas as pd

In [2]:
! cat ../data/longread_models/test_merge/test1.gff

Chr1	test	gene	100	1100	.	+	.	gene_id=gene1;ID=gene1
Chr1	test	transcript	100	1100	.	+	.	gene_id=gene1;ID=transcript1;transcript_id=transcript1;Parent=gene1
Chr1	test	exon	100	200	.	+	.	gene_id=gene1;ID=exon1;transcript_id=transcript1;Parent=transcript1
Chr1	test	exon	1000	1100	.	+	.	gene_id=gene1;ID=exon2;transcript_id=transcript1;Parent=transcript1
Chr1	test	transcript	500	1100	.	+	.	gene_id=gene1;ID=transcript2;transcript_id=transcript2;Parent=gene1
Chr1	test	exon	500	600	.	+	.	gene_id=gene1;ID=exon3;transcript_id=transcript2;Parent=transcript2
Chr1	test	exon	1000	1100	.	+	.	gene_id=gene1;ID=exon4;transcript_id=transcript2;Parent=transcript2

In [3]:
gr = pr.read_gff3('../data/longread_models/test_merge/test1.gff')

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


In [4]:
df = gr.df
df['tissue'] = 'liver'
gr_tag = pr.PyRanges(df)
gr_tag.to_gff3('../data/longread_models/test_merge/test1_tag.gff')

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


### A second gff to merge 

In [5]:
! cat ../data/longread_models/test_merge/test2_tag.gff

Chr1	test	gene	100	1100	.	+	.	gene_id=gene2;ID=gene2;tissue=blood
Chr1	test	transcript	100	1100	.	+	.	gene_id=gene2;ID=transcript1;transcript_id=transcript1;Parent=gene2;tissue=blood
Chr1	test	exon	100	200	.	+	.	gene_id=gene2;ID=exon1;transcript_id=transcript1;Parent=transcript1;tissue=blood
Chr1	test	exon	900	1100	.	+	.	gene_id=gene2;ID=exon2;transcript_id=transcript1;Parent=transcript1;tissue=blood
Chr1	test	transcript	500	600	.	+	.	gene_id=gene2;ID=transcript2;transcript_id=transcript2;Parent=gene2;tissue=blood
Chr1	test	exon	500	600	.	+	.	gene_id=gene2;ID=exon3;transcript_id=transcript2;Parent=transcript2;tissue=blood
Chr1	test	transcript	100	1100	.	+	.	gene_id=gene2;ID=transcript3;transcript_id=transcript3;Parent=gene2;tissue=blood
Chr1	test	exon	100	200	.	+	.	gene_id=gene2;ID=exon4;transcript_id=transcript3;Parent=transcript3;tissue=blood
Chr1	test	exon	500	600	.	+	.	gene_id=gene2;ID=exon5;transcript_id=transcript3;Parent=transcript3;tissue=blood
Chr1	test	exon	1000	1100	.	+	.	ge

Diffs test 1 and 2
- same gene but different names
- transcript1 is has a longer second exon
- transcript2 contains only one exon and it shares it with transcript 2 of test1
- transcript3 combines all exons in test1
- transcript4 same as trnascript1 of test1  
- tissue is different

the two gff were merged with:
`agat_sp_merge_annotations.pl --gff test1_tag.gff --gff test2_tag.gff -o test12_merged.gff`

In [6]:
! cat ../data/longread_models/test_merge/test12_merged.gff

##gff-version 3
Chr1	test	gene	100	1100	.	+	.	ID=gene1;gene_id=gene1,gene2;tissue=liver,blood
Chr1	test	transcript	500	600	.	+	.	ID=IDmodified-transcript-2;Parent=gene1;gene_id=gene2;tissue=blood;transcript_id=transcript2
Chr1	test	exon	500	600	.	+	.	ID=IDmodified-exon-3;Parent=IDmodified-transcript-2;gene_id=gene2;tissue=blood;transcript_id=transcript2
Chr1	test	transcript	100	1100	.	+	.	ID=IDmodified-transcript-1;Parent=gene1;gene_id=gene2;tissue=blood;transcript_id=transcript1
Chr1	test	exon	100	200	.	+	.	ID=IDmodified-exon-1;Parent=IDmodified-transcript-1;gene_id=gene2;tissue=blood;transcript_id=transcript1
Chr1	test	exon	900	1100	.	+	.	ID=IDmodified-exon-2;Parent=IDmodified-transcript-1;gene_id=gene2;tissue=blood;transcript_id=transcript1
Chr1	test	transcript	100	1100	.	+	.	ID=transcript1;Parent=gene1;gene_id=gene1;merged_ID=transcript4;merged_Parent=gene2;merged_gene_id=gene2;merged_tissue=blood;merged_transcript_id=transcript4;tissue=liver;transcript_id=transcript1
Chr1	test	exo

### crete a fake reference to check on IGV

In [8]:
def generate_random_dna_sequence(length):
    """Generate a random DNA sequence of a given length."""
    return ''.join(random.choices('ATCG', k=length))

In [9]:
def write_fasta(filename, sequence, header="random_sequence"):
    """Write the DNA sequence to a FASTA file."""
    with open(filename, 'w') as file:
        file.write(f">{header}\n")
        # Split the sequence into lines of up to 80 characters
        for i in range(0, len(sequence), 80):
            file.write(sequence[i:i+80] + '\n')

In [10]:
# Parameters
sequence_length = 2000
fasta_filename = '../data/longread_models/test_merge/sample_reference.fa'

# Generate the sequence
dna_sequence = generate_random_dna_sequence(sequence_length)

# Write to FASTA file
write_fasta(fasta_filename, dna_sequence, header='Chr1')

### Omogenise names and keep source 
`agat_sp_manage_IDs.pl --type_dependent --gff test12_merged.gff -o test12_merged_fixedID.gff`

In [11]:
! cat ../data/longread_models/test_merge/test12_merged_fixedID.gff

##gff-version 3
Chr1	test	gene	100	1100	.	+	.	ID=gene-1;gene_id=gene1,gene2;tissue=liver,blood
Chr1	test	transcript	500	600	.	+	.	ID=transcript-1;Parent=gene-1;gene_id=gene2;tissue=blood;transcript_id=transcript2
Chr1	test	exon	500	600	.	+	.	ID=exon-1;Parent=transcript-1;gene_id=gene2;tissue=blood;transcript_id=transcript2
Chr1	test	transcript	100	1100	.	+	.	ID=transcript-2;Parent=gene-1;gene_id=gene2;tissue=blood;transcript_id=transcript1
Chr1	test	exon	100	200	.	+	.	ID=exon-2;Parent=transcript-2;gene_id=gene2;tissue=blood;transcript_id=transcript1
Chr1	test	exon	900	1100	.	+	.	ID=exon-3;Parent=transcript-2;gene_id=gene2;tissue=blood;transcript_id=transcript1
Chr1	test	transcript	100	1100	.	+	.	ID=transcript-3;Parent=gene-1;gene_id=gene1;merged_ID=transcript4;merged_Parent=gene2;merged_gene_id=gene2;merged_tissue=blood;merged_transcript_id=transcript4;tissue=liver;transcript_id=transcript1
Chr1	test	exon	100	200	.	+	.	ID=exon-4;Parent=transcript-3;gene_id=gene1;tissue=liver;transcript

In [21]:
merged_fixedID = pr.read_gff3('../data/longread_models/test_merge/test12_merged_fixedID.gff')
merged_fixedID_df = merged_fixedID.df

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


In [22]:
def get_set(x):
    
    """get a set of unique elements from string and ignores whitespaces"""
    if pd.isna(x):
        return np.nan
    
    elements = x.split(',')
    final_list = list(filter(lambda s: s.strip(), elements))
    final_string = ','.join(final_list)            
    
    return final_string
    

In [23]:
def merge_cols(df, merge_into, merge_from):
    
    """retunrs a copy of df where "merge_from" column's elements are merged into "merge_into" """
    """the into colums is removed and from column is substitued """
    
    local_df = df.copy()
    c = local_df[merge_into]+','+local_df[merge_from].fillna('')
    local_df[merge_into] = c.map(get_set)
    local_df = local_df.drop(merge_from, axis=1)

    return local_df   


In [24]:
def transfer_to_child(df, parent, tag_to_transfer):
    
    """This returns a df where a tag to transfer is copied from the parent feature to all child features."""
    """It is useful when merging GFF and not all tags are properly combined."""
    """This assumes no more than one parent per child."""

    local_df = df.copy()
    all_parents = local_df[local_df.Feature == parent].ID.unique()
    for p in all_parents:
        tag_info = local_df[local_df.ID == p][tag_to_transfer].values[0]
        local_df.loc[local_df['Parent'] == p, tag_to_transfer] = tag_info
    
    return local_df
    

In [25]:
merged_fixedID_df =  merge_cols(merged_fixedID_df, 'tissue', 'merged_tissue')
merged_fixedID_df =  merge_cols(merged_fixedID_df, 'gene_id', 'merged_gene_id')
merged_fixedID_df =  merge_cols(merged_fixedID_df, 'transcript_id', 'merged_transcript_id')

In [26]:
# remove manually merged_ID and merged_Parent as they don't have to be merged
merged_fixedID_df = merged_fixedID_df.drop(['merged_ID' ,'merged_Parent'], axis=1)

In [27]:
merged_fixedID_df = transfer_to_child(merged_fixedID_df, 'transcript', 'gene_id')
merged_fixedID_df = transfer_to_child(merged_fixedID_df, 'transcript', 'transcript_id')
merged_fixedID_df = transfer_to_child(merged_fixedID_df, 'transcript', 'tissue')

In [28]:
merged_fixedID_df

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,gene_id,tissue,Parent,transcript_id
0,Chr1,test,gene,99,1100,.,+,.,gene-1,"gene1,gene2","liver,blood",NaN,NaN
1,Chr1,test,transcript,499,600,.,+,.,transcript-1,gene2,blood,gene-1,transcript2
2,Chr1,test,exon,499,600,.,+,.,exon-1,gene2,blood,transcript-1,transcript2
3,Chr1,test,transcript,99,1100,.,+,.,transcript-2,gene2,blood,gene-1,transcript1
4,Chr1,test,exon,99,200,.,+,.,exon-2,gene2,blood,transcript-2,transcript1
5,Chr1,test,exon,899,1100,.,+,.,exon-3,gene2,blood,transcript-2,transcript1
6,Chr1,test,transcript,99,1100,.,+,.,transcript-3,"gene1,gene2","liver,blood",gene-1,"transcript1,transcript4"
7,Chr1,test,exon,99,200,.,+,.,exon-4,"gene1,gene2","liver,blood",transcript-3,"transcript1,transcript4"
8,Chr1,test,exon,999,1100,.,+,.,exon-5,"gene1,gene2","liver,blood",transcript-3,"transcript1,transcript4"
9,Chr1,test,transcript,99,1100,.,+,.,transcript-4,gene2,blood,gene-1,transcript3


In [31]:
rg_final = pr.PyRanges(merged_fixedID_df)
rg_final.to_gff3('../data/longread_models/test_merge/test12_merged_polish.gff')

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


# Todo
To imporove this process, add utility to tag gene_id and transcript_id so they are more easily reconducible to the original models